In [ ]:
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain.tools import Tool
from langchain.chains.sequential import SequentialChain
from langchain.chains.llm import LLMChain
from langchain.chains.transform import TransformChain
import requests
import json
from langchain.prompts import PromptTemplate
from typing import List

def search_kecamatan_api(area: str) -> List[str]:
    url = f"https://algo-api-dev.lionparcel.com/v1/shipment/masterdata/route/origin?keyword={area}"

    try:
        response = requests.get(
            url,
            headers={'Content-Type': 'application/json'}
        )

        response_data = response.json()

        if "data" in response_data and response_data["data"]:
            routes = [item["route"] for item in response_data["data"]]
            return routes
        else:
            return []
    except Exception as e:
        return []

def search_kecamatan(text: str):

    kecamatan_result = json.loads(text['kecamatan_result'])

    pengirim = kecamatan_result['pengirim']

    penerima = kecamatan_result['penerima']

    kecamatan_pengirim = search_kecamatan_api(pengirim['kecamatan'])

    if len(kecamatan_pengirim) > 1:
        kecamatan_pengirim = search_kecamatan_api(f"{pengirim['kelurahan']}, {pengirim['kecamatan']}")

    kecamatan_penerima = search_kecamatan_api(penerima['kecamatan'])

    if len(kecamatan_penerima) > 1:
        kecamatan_penerima = search_kecamatan_api(f"{penerima['kelurahan']}, {penerima['kecamatan']}")

    print(f"{pengirim} => {kecamatan_pengirim}")
    print(f"{penerima} => {kecamatan_penerima}")

    return {
        "kecamatan_result": kecamatan_result,
        "route": {
            "pengirim": kecamatan_pengirim[0],
            "penerima": kecamatan_penerima[0],
        }
    }

def check_tariff_api(text: str):

    shipping_info = json.loads(text['shipping_info'])

    url = f"https://algo-api-dev.lionparcel.com/v5/shipment/elexys/tariff"

    try:
        response = requests.post(
            url,
            headers={'Content-Type': 'application/json'},
            json={
                "weight": shipping_info["paket"]["berat"],
                "length": shipping_info["paket"]["ukuran"]["panjang"],
                "width": shipping_info["paket"]["ukuran"]["lebar"],
                "height": shipping_info["paket"]["ukuran"]["tinggi"],
                "check_tariff_page": False,
                "cod_value": 0,
                "commodity": "GENERAL OTHERS - GENERAL LAINNYA",
                "destination": shipping_info["penerima"]["kecamatan"],
                "origin": shipping_info["pengirim"]["kecamatan"],
                "goods_value": shipping_info["harga_barang"],
                "is_insurance": False,
                "page_type": "dropoff",
                "shipment_type": "DROPOFF" if shipping_info["layanan"] == "Drop" else "PICKUP",
                "is_wood_package": False,
                "is_dfod": False
            }

        )

        response_data = response.json()

        print(f"check tariff => {response_data}")

        return {
            "tariff": response_data["data"]
        }
    except Exception as e:
        return {
            "tariff": []
        }


def format_json_tool(query: str):
    system_content = """
        Gather information and transform into JSON format.

        Follow this format:
        {
            "layanan": "Drop atau Pickup",
            "pengirim": {
                "nama": "string",
                "alamat": "string",
                "no_hp": "string",
                "kecamatan": "string"
            },
            "penerima": {
                "nama": "string",
                "alamat": "string",
                "no_hp": "string",
                "label_alamat": "Rumah atau Kantor",
                "kecamatan": "string"
            },
            "paket": {
                "berat": "number kg",
                "ukuran": {
                    "panjang": "number cm",
                    "lebar": "number cm",
                    "tinggi": "number cm"
                },
                "jumlah_barang": "number"
            },
            "harga_barang": "string dengan format Rpxxx.xxx",
            "tipe_layanan": "COD atau DFOD atau Reguler"
        }
    """

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": query}
    ]

    response = llm.invoke(messages)

    import re

    match = re.search(r"\{.*\}", response.content, re.DOTALL)

    if match:
        json_str = match.group(0)
        try:
            parsed_json = json.loads(json_str)
            return parsed_json
        except json.JSONDecodeError as e:
            return f"Error parsing JSON: {str(e)}"
    else:
        return "No JSON found in response"

from langchain.tools import Tool

prompt_kecamatan = PromptTemplate(
    input_variables=["text"],
    template="""
        Search kelurahan and kecamatan name for both sender and receiver address
        Format:
        {{
            "pengirim": {{
                "kelurahan": "string",
                "kecamatan": "string"
            }},
            "penerima": {{
                "kelurahan": "string",
                "kecamatan": "string"
            }}
        }}
        {text}
"""
)

kecamatan_chain = LLMChain(llm=llm, prompt=prompt_kecamatan, output_key="kecamatan_result")

kecamatan_api_chain = TransformChain(
    input_variables=["kecamatan_result"],
    output_variables=["route"],
    transform=search_kecamatan,
)

prompt_json = PromptTemplate(
    input_variables=["text", "route"],
    template="""
        Gather information and transform into JSON
        Original input: {text}
        Change the pengirim and penerima kecamatan with: {route}
        Format:
        {{
            "layanan": "Drop atau Pickup",
            "pengirim": {{
                "nama": "string",
                "alamat": "string",
                "no_hp": "string",
                "kecamatan": "string",
            }},
            "penerima": {{
                "nama": "string",
                "alamat": "string",
                "no_hp": "string",
                "label_alamat": "Rumah atau Kantor",
                "kecamatan": "string",
            }},
            "paket": {{
                "berat": "number",
                "ukuran": {{
                    "panjang": "number",
                    "lebar": "number",
                    "tinggi": "number"
                }},
                "jumlah_barang": "number"
            }},
            "harga_barang": "number in IDR currency",
            "tipe_layanan": "COD atau DFOD atau Reguler",
            "lokasi_penjemputan": {{
                "latitude": "number",
                "longitude": "number"
            }} (required if layanan = Pickup)
        }}

        Formatting rules:
        - Convert dimensions (length, width, height) to cm (1m = 100cm, 1mm = 0.1cm)
        - Convert weight to kg (1kg = 1000grams)
        - Convert harga barang to kg (1kg = 1000grams)
"""
)

json_chain = LLMChain(llm=llm, prompt=prompt_json, output_key="shipping_info")

check_tariff_api_chain = TransformChain(
    input_variables=["shipping_info"],
    output_variables=["tariff"],
    transform=check_tariff_api,
)

collect_shipping_chain = SequentialChain(
    chains=[kecamatan_chain, kecamatan_api_chain, json_chain],
    input_variables=["text"],
    output_variables=["kecamatan_result", "route", "shipping_info"],
)


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

from langchain.agents import initialize_agent, Tool

tools = [
    Tool(
        name="CollectShipping",
        func=collect_shipping_chain,
        description="""
        Collect shipping info from user input. If user wants to update existing information, use previous data and only update the specified changes."""
    ),
    Tool(
        name="CheckTariff",
        func=check_tariff_api_chain,
        description=""""
        Check tariff using shipping information. Input should be the result from CollectShipping tool.
        Check tariff after shipping info completed or changed. completed is when the sender and receiver kecamatan is knows.

        Return all available tariff with format:
        * Product Name: product
        * Estimated Price: total_estimated_price
        * Estimated Arrival Days: estimated_arrival_in_days
        * Chargeable Weight: chargeable_weight

        Example:
        Product Name: REGPACK
        Estimated Price: Rp 150.000
        Estimated Arrival Days: 1-2 Days
        Chargeable Weight: 1 kg"""
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    # agent="zero-shot-react-description",
    agent="conversational-react-description",
    memory=memory,
    verbose=True,
)

res = agent.run("""
Aku ingin melakukan pengiriman barang.

Pengirim: Hasan Ikhtiarta 081297991631
Alamat: Puri Bojong Lestari II RT 12/17 Blok RR.10, Kel. Pabuaran, Kec. Bojonggede, Kab Bogor 16921

Penerima: Tedja Adhi 0823456789
Alamat: Kantor Lion Parcel Jl Agave, Kel. Kedoya Selatan, Kec. Kebon Jeruk

Berat barang: 1 kg dengan ukuran 1 cm x 2 cm x 3 cm
Jumlah Barang: 2
Harga Barang: Rp150.000
Layanan: Drop
Tipe Layanan: Reguler
""")

print(res)

In [ ]:
res = agent.run("""
Aku ingin merubah berat barang menjadi 10 kg
""")

print(res)

In [ ]:
res = agent.run("""
Aku ingin merubah alamat pengirim dari JL Agasa 10 Bogor Utara
""")

print(res)